# Gender Only UTMB Average Pace Predictions

This notebook predicts average pace using similar runners with the same gender as the query runner. 

In [51]:
import os.path
import datetime
import pandas as pd
import numpy as np
import hashlib
import matplotlib
import matplotlib.pyplot as plt
from scipy.spatial.distance import cosine
from scipy.spatial.distance import cdist
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import squareform, pdist

%matplotlib inline

# Display all columns in Pandas
pd.set_option('display.max_columns', None) 

I aim to produce a challenging but not impossible average pace prediction for each runner. I will first get the runner's non personal best average pace and find the most similar non personal best average pace to that runner. I will then find the personal best average pace of these runners. By average these pb times, or getting the best personal best average pace of these runners, I will be able to produce a predicted average pace. 

In [52]:
# Constants
pace_threshold = 1
#euclidean_distance_threshold = 200

In [53]:
# Read in Split Timings
df = pd.read_csv('../../data/1km_splits_full.csv', encoding = "ISO-8859-1")

In [54]:
len(df)

19579

In [55]:
# Drop unnecessary columns
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'bib','team', 'rank', 'nationality'], axis = 1)

In [56]:
df.head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
0,2017,7efc0cb7e70d7cd9da46a58bbf01af66,Male,Senior Men (23-39),5.281863,NaN,NaN,4.375000,NaN,5.443366,6.483539,9.900000,4.980392,8.205502,4.044218,9.433333,5.237037,5.646341,NaN,NaN,10.649660,6.691441,6.487179,11.881481,5.323333,5.707143,NaN,NaN,8.030973,NaN,NaN,6.102041,NaN,8.604167,NaN,7.747126,NaN,NaN,7.788288,NaN,NaN,9.016667,5.236486,1141.900000,NaN,7.012357
1,2017,1e4ba9a22b3fecb9136ecfcf7f470868,Male,Senior Men (23-39),5.147059,NaN,NaN,4.519737,NaN,5.501618,6.510288,10.100000,4.970588,8.098706,4.047619,9.429167,5.255556,5.743902,NaN,NaN,11.159864,6.887387,6.810897,12.225926,5.376667,5.800000,NaN,NaN,8.781711,NaN,NaN,5.768707,NaN,8.729167,NaN,7.235632,NaN,NaN,7.869369,NaN,NaN,9.300000,5.072072,1156.983333,15.083333,7.097568
2,2017,0d928f7482046900e19dba217d7befa5,Male,Senior Men (23-39),5.544118,NaN,NaN,4.372807,NaN,5.500000,6.981481,10.769697,5.689542,8.812298,4.452381,10.020833,5.629630,6.292683,NaN,NaN,11.411565,6.979730,6.519231,NaN,11.085000,5.870238,NaN,NaN,8.631268,NaN,NaN,5.323129,NaN,9.222222,NaN,7.497126,NaN,NaN,7.918919,NaN,NaN,9.804762,4.772523,1193.000000,51.100000,7.352225
3,2017,93f20063908ba5d48724dcf31740a52a,Male,Senior Men (23-39),5.283088,NaN,NaN,4.379386,NaN,5.441748,6.489712,11.700000,5.741830,9.105178,4.812925,10.220833,5.677778,6.516260,NaN,NaN,12.340136,7.243243,7.243590,13.255556,5.170000,5.839286,NaN,NaN,8.510324,NaN,NaN,6.119048,NaN,9.104167,NaN,7.370690,NaN,NaN,7.585586,NaN,NaN,9.238095,4.808559,1203.650000,61.750000,7.466542
4,2017,7479fb90beffad048c5f772ba1915b4e,Male,Senior Men (23-39),5.145833,NaN,NaN,4.245614,NaN,5.582524,6.615226,9.887879,4.973856,8.249191,3.901361,8.366667,5.162963,6.024390,NaN,NaN,10.870748,7.063063,6.448718,11.896296,5.868333,7.513095,NaN,NaN,12.036873,NaN,NaN,4.996599,NaN,8.180556,NaN,6.422414,NaN,NaN,8.414414,NaN,NaN,10.057143,5.141892,1211.633333,69.733333,7.211069


In [57]:
df.shape

(19579, 46)

In [58]:
df.groupby('category').count()

,Year,name_mask,gender,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
category,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Masters Men 1 (40-49),7881,7881,7881,5994,560,954,5990,801,7749,7435,7379,7389,7116,7098,6886,6834,6869,1697,816,5475,6215,6176,5779,5764,5339,1816,540,2726,1811,534,2722,4254,714,4262,714,3707,524,712,3705,523,711,4408,4931,4930,7877
Masters Men 2 (50-59),3393,3393,3393,2592,220,376,2588,373,3312,3145,3116,3119,2966,2944,2848,2814,2838,675,330,2172,2462,2438,2230,2220,2004,736,183,974,733,183,970,1608,239,1606,239,1403,173,235,1398,172,236,1633,1805,1805,3388
Masters Men 3 (60-69),520,520,520,371,43,61,370,74,502,458,453,454,423,417,388,387,384,70,35,281,309,304,261,254,216,100,23,76,100,23,77,180,13,178,13,148,22,13,148,21,13,162,183,183,520
Masters Men 4 (70-79),29,29,29,23,1,4,23,2,28,27,25,26,23,23,21,21,20,2,1,14,15,15,12,12,7,3,0,2,3,0,2,5,0,4,0,4,0,0,4,0,0,4,4,4,29
Masters Women 1 (40-49),719,719,719,560,51,69,560,81,702,668,660,665,633,625,607,601,602,166,72,466,537,530,474,473,431,140,29,240,140,29,239,341,61,343,61,309,27,61,310,27,61,370,397,397,717
Masters Women 2 (50-59),287,287,287,220,15,37,220,26,278,261,259,258,239,237,227,228,224,51,23,171,191,187,169,168,150,60,12,69,60,12,69,117,20,116,20,100,12,20,101,12,20,121,133,133,285
Masters Women 3 (60-69),26,26,26,21,0,3,21,2,24,21,22,21,18,18,17,16,16,8,5,7,12,12,9,9,8,3,1,4,3,1,4,8,0,8,0,7,1,0,7,1,0,7,8,8,26
Senior Men (23-39),6042,6042,6042,4555,501,700,4555,674,5943,5737,5703,5706,5511,5515,5366,5315,5374,1325,639,4360,4955,4917,4701,4687,4354,1493,367,2247,1481,355,2248,3418,586,3423,586,3046,344,580,3044,344,580,3628,3972,3967,6036
Senior Women (23-39),646,646,646,519,31,69,518,51,630,614,606,607,585,579,561,558,563,156,70,440,498,493,460,457,421,134,33,237,134,33,234,326,66,324,66,284,32,66,285,32,66,348,380,380,645


In [59]:
df[(df["AVG_Pace"] <12) & (df.category == 'Masters Women 2 (50-59)')]


,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
2471,2017,6edbc123d2c45fa61b34a1f47eae4a2c,Female,Masters Women 2 (50-59),10.267157,NaN,NaN,9.618421,NaN,11.441748,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.442442
2485,2017,dd9e346cc77bdb575a062caf1e045922,Female,Masters Women 2 (50-59),11.279412,NaN,NaN,10.745614,NaN,12.241100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.422042
2494,2017,086c45154e617591ad9887ce3358be2d,Female,Masters Women 2 (50-59),10.833333,NaN,NaN,11.392544,NaN,13.122977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.782952
2595,2016,36da23ac15e20c3b3e35b879229da995,Female,Masters Women 2 (50-59),7.949755,NaN,NaN,5.844298,NaN,8.763754,9.913580,13.748485,7.372549,10.580906,14.908163,13.258333,7.566667,8.939024,18.055556,NaN,14.758503,8.916667,8.650641,15.911111,6.843333,8.625000,NaN,NaN,11.520649,NaN,NaN,8.564626,16.606918,NaN,8.194444,NaN,17.140693,NaN,NaN,14.343434,NaN,NaN,8.497748,1753.933333,433.900000,11.018994
5066,2016,518c744fb20eb7d4c43afd66c1410b57,Female,Masters Women 2 (50-59),10.808824,NaN,NaN,8.978070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.893447
5079,2016,8cb28bf1befd988376fe7152453f08e9,Female,Masters Women 2 (50-59),11.164216,NaN,NaN,11.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.207108
5081,2016,85d4d22c35b487dcb6fa7c4fcfca9ae3,Female,Masters Women 2 (50-59),13.436275,NaN,NaN,9.083333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.259804
7405,2015,cbf15f23ef5ead61ed9491f0e92aa9b8,Female,Masters Women 2 (50-59),9.091912,NaN,NaN,7.708333,NaN,9.885113,14.232510,18.296970,9.911765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.521101
7546,2015,ea16550cfbbfd6cd4327356dc80126e6,Female,Masters Women 2 (50-59),9.000000,NaN,NaN,9.063596,NaN,11.915858,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.993151
7559,2015,7c2c1575d30df2b22d3662dc49bc41bb,Female,Masters Women 2 (50-59),10.167892,NaN,NaN,8.837719,NaN,11.711974,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.239195


In [60]:
df.loc[1613]

Year                                                        2017
name_mask                       4db319a671ab6e2b099df28df89f85c4
gender                                                      Male
category                                   Masters Men 1 (40-49)
Chamonix to Delevret                                      8.6826
Chamonix to La Charme                                        NaN
Chamonix to St-Gervais                                       NaN
Delevret to St-Gervais                                   8.11842
La Charme to St-Gervais                                      NaN
St-Gervais to Contamines                                 10.7799
Contamines to La Balme                                   14.7181
La Balme to Bonhomme                                     22.7212
Bonhomme to Chapieux                                     10.8431
Chapieux to Col Seigne                                   17.9725
Col Seigne to Lac Combal                                 10.3231
Lac Combal to Mt-Favre   

In [61]:
# Filter out NaN times
df = df.dropna(subset=['time'])
# Sort by name (first) and year (second)
# Sorting by year, since it might make sense to later only generate prediction for latest non-pb time
df.sort_values(['name_mask', 'Year'], ascending=[True, False], inplace=True)
# Show sample of data
df.head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
11228,2013,0001fff82652ce3818504f26771297a3,Male,Senior Men (23-39),7.925245,NaN,NaN,6.611842,NaN,10.315534,10.477366,15.990909,10.611111,15.563107,9.758503,22.025000,11.170370,13.402439,NaN,NaN,33.187075,12.585586,11.990385,23.470370,10.875000,13.505952,21.299663,NaN,NaN,15.997354,NaN,NaN,27.305031,NaN,18.133333,NaN,27.787879,NaN,NaN,18.045455,NaN,NaN,18.114865,2586.933333,1351.983333,16.089557
3623,2016,0002b9e98b5761bb4aa0960616f45ca6,Male,Masters Men 1 (40-49),8.459559,NaN,NaN,7.653509,NaN,11.849515,17.141975,18.469697,12.617647,16.720065,24.044218,17.337500,11.022222,13.654472,66.444444,NaN,18.125850,14.306306,12.666667,23.518519,11.310000,17.688095,NaN,NaN,19.547198,NaN,NaN,14.591837,28.166667,NaN,11.530556,NaN,19.720779,NaN,NaN,14.439394,NaN,NaN,11.335586,2646.166667,1326.133333,17.694491
5634,2015,000bf78327bdfa8c6298fc9a76921305,Male,Masters Men 1 (40-49),7.971814,NaN,NaN,6.657895,NaN,9.449838,12.952675,17.209091,10.323529,14.173139,21.414966,18.554167,10.270370,11.796748,35.777778,23.616667,NaN,12.972973,12.865385,23.362963,11.135000,12.634524,NaN,NaN,17.690265,NaN,NaN,12.363946,20.327044,NaN,12.508333,NaN,17.738095,NaN,NaN,16.020202,NaN,NaN,10.786036,2323.750000,1054.500000,15.222938
7880,2014,000d01e06e9abfe51cdb08ff2567f125,Male,Senior Men (23-39),7.726716,NaN,NaN,6.125000,NaN,8.590615,10.203704,14.993939,7.905229,11.296117,7.071429,15.108333,8.466667,10.317073,NaN,NaN,21.159864,10.364865,10.740385,18.259259,9.588333,11.288095,NaN,NaN,15.786136,NaN,NaN,11.030612,18.198113,NaN,13.405556,NaN,18.824675,NaN,NaN,18.515152,NaN,NaN,13.027027,2021.466667,809.733333,12.416371
15238,2009,00156070bc3ed3fd8b6b87dabd51c2fc,Male,Senior Men (23-39),NaN,NaN,8.779845,NaN,NaN,8.166667,10.508230,15.206061,8.872549,12.690939,7.278912,14.308333,7.914815,8.349593,NaN,NaN,24.034014,10.572072,11.000000,22.844444,8.900000,13.594048,17.599327,NaN,NaN,12.955026,NaN,NaN,18.968553,NaN,13.488889,NaN,20.811688,NaN,NaN,16.530303,NaN,NaN,16.921171,2193.000000,899.700000,13.491108


# Filter out runners who only ran one race

Runner's have ran at least 2 races to be included in the case base, ie. a non personal best time and a personal best time

In [62]:
# Count number of races per runner
v = df.name_mask.value_counts()
# Get runners who ran more than one race
multi_race_runners = v.index[v.gt(1)]
# Filter by multi_race_runners
df = df[df.name_mask.isin(multi_race_runners)]

In [63]:
df.head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
8150,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),9.541667,NaN,NaN,8.592105,NaN,10.600324,12.399177,17.733333,9.107843,14.792880,8.591837,17.929167,8.622222,10.630081,NaN,NaN,25.088435,11.418919,13.769231,21.262963,8.951667,12.495238,NaN,NaN,18.377581,NaN,NaN,15.748299,22.459119,NaN,13.636111,NaN,22.088745,NaN,NaN,18.590909,NaN,NaN,7.968468,2312.483333,1100.750000,14.183180
10507,2013,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501
13323,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),10.694853,NaN,NaN,10.660088,NaN,11.444984,14.790123,18.772727,10.356209,14.632686,9.476190,17.833333,11.033333,13.252033,NaN,NaN,25.986395,11.813063,16.256410,25.833333,15.586667,17.473810,NaN,21.075426,NaN,NaN,16.559829,NaN,21.588050,NaN,11.688889,NaN,NaN,10.767196,NaN,NaN,11.762452,NaN,NaN,2581.333333,1344.616667,15.188612
12954,2011,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),8.261029,NaN,NaN,7.385965,NaN,9.022654,11.744856,15.790909,8.666667,13.215210,8.047619,16.012500,9.270370,11.483740,NaN,NaN,30.527211,10.997748,12.743590,21.881481,13.003333,16.636905,NaN,21.569343,NaN,NaN,19.384615,NaN,20.207547,NaN,12.561111,NaN,NaN,11.484127,NaN,NaN,10.839080,NaN,NaN,2370.466667,1133.750000,13.945114
18340,2008,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),NaN,4.723913,NaN,NaN,8.472222,9.040453,12.572016,16.218182,9.852941,15.737864,11.806122,21.025000,8.951852,8.869919,NaN,NaN,25.017007,19.596847,13.961538,26.729630,10.395000,14.508333,27.432660,NaN,NaN,21.005291,NaN,NaN,24.845912,NaN,17.430556,NaN,30.506494,NaN,NaN,15.212121,NaN,NaN,18.905405,2707.316667,1450.333333,16.367387


# Merge

In [64]:
# Get Personal Best
pb = df[df.time.groupby(df.name_mask).apply(lambda x: x == x.min())]
# Drop Year and category to avoid duplication on merge
pb = pb.drop(['Year', 'category'], axis = 1)
# Get Non-Personal Best Times
npb = df[~df.time.groupby(df.name_mask).apply(lambda x: x == x.min())]

In [65]:
pb.head()

,name_mask,gender,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
10507,003cc0338235bb1862eb7b9bdd31ec9c,Male,9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501
12954,006db776bb4048de7f357c632a8778ac,Male,8.261029,NaN,NaN,7.385965,NaN,9.022654,11.744856,15.790909,8.666667,13.215210,8.047619,16.012500,9.270370,11.483740,NaN,NaN,30.527211,10.997748,12.743590,21.881481,13.003333,16.636905,NaN,21.569343,NaN,NaN,19.384615,NaN,20.207547,NaN,12.561111,NaN,NaN,11.484127,NaN,NaN,10.83908,NaN,NaN,2370.466667,1133.750000,13.945114
8431,00a7ade381ebb6d25e1aacb0bc7129e9,Male,8.466912,NaN,NaN,7.383772,NaN,8.750809,12.213992,16.269697,10.627451,14.220065,9.401361,17.579167,10.533333,10.540650,NaN,NaN,29.227891,11.637387,15.320513,23.751852,11.983333,14.132143,NaN,NaN,20.764012,NaN,NaN,15.326531,26.301887,NaN,16.358333,NaN,23.008658,NaN,NaN,18.101010,NaN,NaN,11.027027,2456.833333,1245.100000,15.121991
15237,00b39183bebaffe426e376f7efc8ae1b,Male,NaN,NaN,7.547287,NaN,NaN,9.362460,12.662551,16.833333,11.153595,13.598706,8.969388,17.229167,9.374074,10.439024,NaN,NaN,21.806122,10.963964,12.903846,19.774074,9.646667,11.486905,16.872054,NaN,NaN,15.801587,NaN,NaN,20.562893,NaN,13.461111,NaN,19.062771,NaN,NaN,14.969697,NaN,NaN,12.225225,2192.983333,899.683333,13.769848
6180,011b597a8c4121461b659f50dbd19d5a,Male,8.411765,NaN,NaN,7.717105,NaN,10.508091,12.713992,19.566667,11.323529,15.192557,23.520408,19.091667,11.370370,12.223577,29.333333,27.775,NaN,15.608108,16.535256,24.925926,11.051667,14.361905,NaN,NaN,21.212389,NaN,NaN,13.040816,25.194969,NaN,12.630556,NaN,21.954545,NaN,NaN,17.929293,NaN,NaN,12.461712,2595.100000,1325.850000,16.626208


In [66]:
npb.head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
8150,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),9.541667,NaN,NaN,8.592105,NaN,10.600324,12.399177,17.733333,9.107843,14.792880,8.591837,17.929167,8.622222,10.630081,NaN,NaN,25.088435,11.418919,13.769231,21.262963,8.951667,12.495238,NaN,NaN,18.377581,NaN,NaN,15.748299,22.459119,NaN,13.636111,NaN,22.088745,NaN,NaN,18.590909,NaN,NaN,7.968468,2312.483333,1100.750000,14.183180
13323,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),10.694853,NaN,NaN,10.660088,NaN,11.444984,14.790123,18.772727,10.356209,14.632686,9.476190,17.833333,11.033333,13.252033,NaN,NaN,25.986395,11.813063,16.256410,25.833333,15.586667,17.473810,NaN,21.075426,NaN,NaN,16.559829,NaN,21.588050,NaN,11.688889,NaN,NaN,10.767196,NaN,NaN,11.762452,NaN,NaN,2581.333333,1344.616667,15.188612
18340,2008,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),NaN,4.723913,NaN,NaN,8.472222,9.040453,12.572016,16.218182,9.852941,15.737864,11.806122,21.025000,8.951852,8.869919,NaN,NaN,25.017007,19.596847,13.961538,26.729630,10.395000,14.508333,27.432660,NaN,NaN,21.005291,NaN,NaN,24.845912,NaN,17.430556,NaN,30.506494,NaN,NaN,15.212121,NaN,NaN,18.905405,2707.316667,1450.333333,16.367387
3763,2016,00a7ade381ebb6d25e1aacb0bc7129e9,Male,Masters Men 1 (40-49),8.912990,NaN,NaN,7.186404,NaN,10.422330,13.958848,19.400000,11.287582,16.563107,26.071429,16.337500,10.370370,11.008130,48.0,NaN,22.217687,14.506757,17.227564,21.659259,11.713333,14.100000,NaN,NaN,22.088496,NaN,NaN,15.302721,25.490566,NaN,13.372222,NaN,25.939394,NaN,NaN,20.959596,NaN,NaN,15.957207,2703.916667,1383.883333,17.602140
17546,2008,00b39183bebaffe426e376f7efc8ae1b,Male,Masters Men 1 (40-49),NaN,NaN,NaN,NaN,29.197222,10.140777,13.462963,17.600000,11.225490,14.611650,11.180272,17.779167,10.611111,10.662602,NaN,NaN,22.261905,12.840090,13.230769,20.633333,10.486667,11.557143,17.387205,NaN,NaN,15.240741,NaN,NaN,20.226415,NaN,13.408333,NaN,19.309524,NaN,NaN,11.808081,NaN,NaN,9.322072,2255.450000,998.466667,14.964501


In [67]:
# Merge to get non-pb runs with personal best time (with time, timediff and AVG_Pace)
merge = npb.merge(pb,how='left',on='name_mask',suffixes=('', '_pb'))

In [68]:
#merge = merge.groupby('name_mask').head(1)

In [69]:
merge.head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,gender_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
0,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),9.541667,NaN,NaN,8.592105,NaN,10.600324,12.399177,17.733333,9.107843,14.792880,8.591837,17.929167,8.622222,10.630081,NaN,NaN,25.088435,11.418919,13.769231,21.262963,8.951667,12.495238,NaN,NaN,18.377581,NaN,NaN,15.748299,22.459119,NaN,13.636111,NaN,22.088745,NaN,NaN,18.590909,NaN,NaN,7.968468,2312.483333,1100.750000,14.183180,Male,9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501
1,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),10.694853,NaN,NaN,10.660088,NaN,11.444984,14.790123,18.772727,10.356209,14.632686,9.476190,17.833333,11.033333,13.252033,NaN,NaN,25.986395,11.813063,16.256410,25.833333,15.586667,17.473810,NaN,21.075426,NaN,NaN,16.559829,NaN,21.588050,NaN,11.688889,NaN,NaN,10.767196,NaN,NaN,11.762452,NaN,NaN,2581.333333,1344.616667,15.188612,Male,9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501
2,2008,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),NaN,4.723913,NaN,NaN,8.472222,9.040453,12.572016,16.218182,9.852941,15.737864,11.806122,21.025000,8.951852,8.869919,NaN,NaN,25.017007,19.596847,13.961538,26.729630,10.395000,14.508333,27.432660,NaN,NaN,21.005291,NaN,NaN,24.845912,NaN,17.430556,NaN,30.506494,NaN,NaN,15.212121,NaN,NaN,18.905405,2707.316667,1450.333333,16.367387,Male,8.261029,NaN,NaN,7.385965,NaN,9.022654,11.744856,15.790909,8.666667,13.215210,8.047619,16.012500,9.270370,11.483740,NaN,NaN,30.527211,10.997748,12.743590,21.881481,13.003333,16.636905,NaN,21.5693

In [70]:
first_df = len(merge)

In [71]:
merge.loc[1613]

Year                                                           2015
name_mask                          fbac5ac0b66892696a502c9f3de3ee0e
gender                                                         Male
category                                          Youth Men (20-22)
Chamonix to Delevret                                        7.48897
Chamonix to La Charme                                           NaN
Chamonix to St-Gervais                                          NaN
Delevret to St-Gervais                                      5.99342
La Charme to St-Gervais                                         NaN
St-Gervais to Contamines                                    7.80583
Contamines to La Balme                                      9.45885
La Balme to Bonhomme                                        14.3242
Bonhomme to Chapieux                                        8.26471
Chapieux to Col Seigne                                      11.6942
Col Seigne to Lac Combal                        

In [72]:
merge.groupby('category').count()

,Year,name_mask,gender,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,gender_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
category,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Masters Men 1 (40-49),690,690,690,484,65,96,484,88,687,688,689,690,689,688,687,674,689,144,77,612,687,686,689,690,690,289,122,279,289,122,279,632,57,633,57,510,122,57,510,122,57,568,690,690,690,690,509,56,113,509,60,685,689,686,689,688,690,687,677,690,106,59,631,685,682,689,690,690,308,92,290,308,92,288,610,77,613,77,521,92,76,521,92,77,598,690,690,690
Masters Men 2 (50-59),290,290,290,233,14,30,233,22,290,289,290,290,289,290,288,288,288,79,39,250,289,290,290,290,290,99,42,148,99,42,149,255,34,256,34,213,42,34,213,42,34,248,290,290,290,290,167,33,67,168,51,286,289,286,288,290,290,290,283,290,32,24,265,290,283,290,290,290,164,39,87,164,39,86,268,22,268,22,228,39,22,229,39,22,251,290,290,290
Masters Men 3 (60-69),31,31,31,26,5,0,25,4,31,31,31,31,31,31,31,31,31,7,4,26,31,31,31,31,31,8,6,17,8,6,17,26,5,26,5,20,6,5,20,6,5,25,31,31,31,31,12,0,15,12,4,31,31,30,31,31,31,31,31,31,0,0,31,31,31,31,31,31,23,6,2,23,6,2,30,1,30,1,24,6,1,24,6,1,25,31,31,31
Masters Men 4 (70-79),1,1,1,0,0,1,0,0,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,1,1,1,1,1,0,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,1,1,1,1
Masters Women 1 (40-49),46,46,46,35,3,6,35,4,46,46,46,46,46,46,45,46,46,16,9,36,46,46,46,46,46,16,3,27,16,3,27,43,3,43,3,40,3,3,40,3,3,43,46,46,46,46,38,1,7,38,1,46,46,46,46,46,46,46,46,46,10,3,43,46,46,46,46,46,13,6,27,13,6,27,36,10,36,10,30,6,10,30,6,10,40,46,46,46
Masters Women 2 (50-59),14,14,14,11,1,2,11,0,14,14,14,14,14,14,14,14,14,4,1,13,14,14,14,14,14,5,2,7,5,2,7,12,2,12,2,10,2,2,10,2,2,12,14,14,14,14,12,0,1,12,1,14,14,14,14,14,14,14,14,14,0,0,14,14,14,14,14,14,7,2,5,7,2,5,13,1,13,1,11,2,1,11,2,1,12,14,14,14
Masters Women 3 (60-69),2,2,2,2,0,0,2,0,2,2,2,2,2,2,2,2,2,1,1,1,2,2,2,2,2,0,0,2,0,0,2,2,0,2,0,2,0,0,2,0,0,2,2,2,2,2,2,0,0,2,0,2,2,2,2,2,2,2,2,2,0,0,2,2,2,2,2,2,2,0,0,2,0,0,2,0,2,0,2,0,0,2,0,0,2,2,2,2
Senior Men (23-39),501,501,501,318,59,89,319,84,497,499,500,500,499,499,

In [73]:
#df = df[~df["AVG_Pace"] > 12 & df.category == 'Masters Men 3 (60-69)']

merge[(merge["AVG_Pace"] <12) & (merge.category == 'Masters Women 2 (50-59)')]



#df[(df["AVG_Pace"] <12) & (df.category == 'Masters Women 2 (50-59)')]


,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,gender_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
347,2016,36da23ac15e20c3b3e35b879229da995,Female,Masters Women 2 (50-59),7.949755,NaN,NaN,5.844298,NaN,8.763754,9.91358,13.748485,7.372549,10.580906,14.908163,13.258333,7.566667,8.939024,18.055556,NaN,14.758503,8.916667,8.650641,15.911111,6.843333,8.625,NaN,NaN,11.520649,NaN,NaN,8.564626,16.606918,NaN,8.194444,NaN,17.140693,NaN,NaN,14.343434,NaN,NaN,8.497748,1753.933333,433.9,11.018994,Female,7.545343,NaN,NaN,6.116228,NaN,8.108414,9.133745,14.109091,7.745098,10.985437,6.323129,12.966667,7.181481,8.353659,NaN,NaN,16.510204,9.153153,8.86859,16.151852,7.035,8.82619,NaN,NaN,11.446903,NaN,NaN,7.55102,15.613208,NaN,7.3,NaN,15.718615,NaN,NaN,11.636364,NaN,NaN,9.177928,1667.866667,456.133333,10.148222


##### Runners with no similar runners

In [76]:
merge = merge.reset_index(drop=True)

In [77]:
merge.loc[1606]

Year                                                           2008
name_mask                          fad726596298ecf4c3f78b8c0134dc1d
gender                                                         Male
category                                      Masters Men 1 (40-49)
Chamonix to Delevret                                            NaN
Chamonix to La Charme                                           NaN
Chamonix to St-Gervais                                          NaN
Delevret to St-Gervais                                          NaN
La Charme to St-Gervais                                      27.775
St-Gervais to Contamines                                    8.22492
Contamines to La Balme                                       10.465
La Balme to Bonhomme                                         15.197
Bonhomme to Chapieux                                        9.32026
Chapieux to Col Seigne                                      12.2816
Col Seigne to Lac Combal                        

In [78]:
cats = merge['category']
gen = merge['gender']
comb = [cats, gen]
comb = pd.concat(comb, axis=1)

In [79]:
# Update pb and npb to use merged indices
# pb does not contain name_mask, category or year
pb = merge.loc[:, merge.columns.str.contains('pb')]
npb = merge.loc[:, ~merge.columns.str.contains('pb')]

In [80]:
merge.head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,gender_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
0,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),9.541667,NaN,NaN,8.592105,NaN,10.600324,12.399177,17.733333,9.107843,14.792880,8.591837,17.929167,8.622222,10.630081,NaN,NaN,25.088435,11.418919,13.769231,21.262963,8.951667,12.495238,NaN,NaN,18.377581,NaN,NaN,15.748299,22.459119,NaN,13.636111,NaN,22.088745,NaN,NaN,18.590909,NaN,NaN,7.968468,2312.483333,1100.750000,14.183180,Male,9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501
1,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),10.694853,NaN,NaN,10.660088,NaN,11.444984,14.790123,18.772727,10.356209,14.632686,9.476190,17.833333,11.033333,13.252033,NaN,NaN,25.986395,11.813063,16.256410,25.833333,15.586667,17.473810,NaN,21.075426,NaN,NaN,16.559829,NaN,21.588050,NaN,11.688889,NaN,NaN,10.767196,NaN,NaN,11.762452,NaN,NaN,2581.333333,1344.616667,15.188612,Male,9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501
2,2008,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),NaN,4.723913,NaN,NaN,8.472222,9.040453,12.572016,16.218182,9.852941,15.737864,11.806122,21.025000,8.951852,8.869919,NaN,NaN,25.017007,19.596847,13.961538,26.729630,10.395000,14.508333,27.432660,NaN,NaN,21.005291,NaN,NaN,24.845912,NaN,17.430556,NaN,30.506494,NaN,NaN,15.212121,NaN,NaN,18.905405,2707.316667,1450.333333,16.367387,Male,8.261029,NaN,NaN,7.385965,NaN,9.022654,11.744856,15.790909,8.666667,13.215210,8.047619,16.012500,9.270370,11.483740,NaN,NaN,30.527211,10.997748,12.743590,21.881481,13.003333,16.636905,NaN,21.5693

In [81]:
merge[merge.category == 'Masters Women 2 (50-59)']

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,gender_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
113,2015,1320ec5e4c7a443d78f71103526fc704,Female,Masters Women 2 (50-59),10.644608,NaN,NaN,9.164474,NaN,11.508091,12.911523,20.460606,11.258170,15.555016,24.010204,24.725000,11.185185,18.715447,10.592593,28.2,NaN,14.851351,13.451923,26.985185,11.031667,15.626190,NaN,NaN,19.264012,NaN,NaN,13.520408,20.616352,NaN,10.550000,NaN,20.690476,NaN,NaN,17.156566,NaN,NaN,11.671171,2622.816667,1353.566667,16.173849,Female,8.974265,NaN,NaN,7.760965,NaN,9.928803,12.845679,19.306061,10.006536,16.221683,9.071429,19.350000,10.466667,12.544715,NaN,NaN,28.346939,12.727477,14.567308,24.359259,10.616667,12.571429,NaN,NaN,18.262537,NaN,NaN,17.734694,NaN,20.586806,NaN,23.600575,NaN,NaN,17.810811,NaN,NaN,23.142857,14.245495,2494.933333,1353.033333,15.627069
114,2013,13213e63d3265f0750495f08ab12d901,Female,Masters Women 2 (50-59),9.515931,NaN,NaN,8.736842,NaN,10.922330,11.903292,18.412121,11.101307,13.843042,10.523810,16.550000,9.881481,11.918699,NaN,NaN,23.275510,13.722973,14.701923,21.314815,11.460000,13.453571,17.031987,NaN,NaN,16.849206,NaN,NaN,24.349057,NaN,15.975000,NaN,18.474026,NaN,NaN,13.616162,NaN,NaN,8.671171,2358.233333,1123.283333,14.425177,Female,9.609069,NaN,NaN,9.385965,NaN,9.663430,11.991770,16.154545,9.983660,12.760518,8.408163,15.145833,9.540741,11.121951,NaN,NaN,21.034014,10.822072,11.253205,18.418519,12.285000,13.139286,NaN,17.222628,NaN,NaN,18.901709,NaN,21.861635,NaN,13.383333,NaN,NaN,12.624339,NaN,NaN,13.045977,NaN,NaN,2264.133333,1027.416667,13.380755
304,2016,30f977102cb8714bf2b596874ef040d2,Female,Masters Women 2 (50-59),9.572304,NaN,NaN,8.432018,NaN,11.436893,16.444444,20.839394,10.205882,18.113269,21.234694,21.037500,10.081481,13.101626,55.481481,NaN,20.336735,12.605856,12.746795,23.814815,10.713333,15.880952,NaN,NaN,22.346608,NaN,NaN,16.455782,25.698113,NaN,11.341667,NaN,21.900433,NaN,NaN,16.929293,NaN,NaN,10.472973,2670.966667,1350.933333,17.488974,Female,9.156863,NaN,NaN,7.513158,NaN,8.949838,12.718107,17.206061,7.424837,13.823625,6.894558,13.316667,10.548148,8.434959,NaN,NaN,21.061224,10.44819

In [82]:
merge.head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,gender_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
0,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),9.541667,NaN,NaN,8.592105,NaN,10.600324,12.399177,17.733333,9.107843,14.792880,8.591837,17.929167,8.622222,10.630081,NaN,NaN,25.088435,11.418919,13.769231,21.262963,8.951667,12.495238,NaN,NaN,18.377581,NaN,NaN,15.748299,22.459119,NaN,13.636111,NaN,22.088745,NaN,NaN,18.590909,NaN,NaN,7.968468,2312.483333,1100.750000,14.183180,Male,9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501
1,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),10.694853,NaN,NaN,10.660088,NaN,11.444984,14.790123,18.772727,10.356209,14.632686,9.476190,17.833333,11.033333,13.252033,NaN,NaN,25.986395,11.813063,16.256410,25.833333,15.586667,17.473810,NaN,21.075426,NaN,NaN,16.559829,NaN,21.588050,NaN,11.688889,NaN,NaN,10.767196,NaN,NaN,11.762452,NaN,NaN,2581.333333,1344.616667,15.188612,Male,9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501
2,2008,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),NaN,4.723913,NaN,NaN,8.472222,9.040453,12.572016,16.218182,9.852941,15.737864,11.806122,21.025000,8.951852,8.869919,NaN,NaN,25.017007,19.596847,13.961538,26.729630,10.395000,14.508333,27.432660,NaN,NaN,21.005291,NaN,NaN,24.845912,NaN,17.430556,NaN,30.506494,NaN,NaN,15.212121,NaN,NaN,18.905405,2707.316667,1450.333333,16.367387,Male,8.261029,NaN,NaN,7.385965,NaN,9.022654,11.744856,15.790909,8.666667,13.215210,8.047619,16.012500,9.270370,11.483740,NaN,NaN,30.527211,10.997748,12.743590,21.881481,13.003333,16.636905,NaN,21.5693

In [83]:
pb.head()

,gender_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb
0,Male,9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501
1,Male,9.375000,NaN,NaN,6.822368,NaN,9.946602,11.195473,16.830303,8.232026,13.820388,7.568027,17.150000,8.537037,10.073171,NaN,NaN,22.479592,10.628378,10.935897,23.677778,7.725000,12.082143,13.244108,NaN,NaN,12.547619,NaN,NaN,19.603774,NaN,15.133333,NaN,22.216450,NaN,NaN,18.792929,NaN,NaN,15.130631,2183.316667,948.366667,13.489501
2,Male,8.261029,NaN,NaN,7.385965,NaN,9.022654,11.744856,15.790909,8.666667,13.215210,8.047619,16.012500,9.270370,11.483740,NaN,NaN,30.527211,10.997748,12.743590,21.881481,13.003333,16.636905,NaN,21.569343,NaN,NaN,19.384615,NaN,20.207547,NaN,12.561111,NaN,NaN,11.484127,NaN,NaN,10.83908,NaN,NaN,2370.466667,1133.750000,13.945114
3,Male,8.466912,NaN,NaN,7.383772,NaN,8.750809,12.213992,16.269697,10.627451,14.220065,9.401361,17.579167,10.533333,10.540650,NaN,NaN,29.227891,11.637387,15.320513,23.751852,11.983333,14.132143,NaN,NaN,20.764012,NaN,NaN,15.326531,26.301887,NaN,16.358333,NaN,23.008658,NaN,NaN,18.101010,NaN,NaN,11.027027,2456.833333,1245.100000,15.121991
4,Male,NaN,NaN,7.547287,NaN,NaN,9.362460,12.662551,16.833333,11.153595,13.598706,8.969388,17.229167,9.374074,10.439024,NaN,NaN,21.806122,10.963964,12.903846,19.774074,9.646667,11.486905,16.872054,NaN,NaN,15.801587,NaN,NaN,20.562893,NaN,13.461111,NaN,19.062771,NaN,NaN,14.969697,NaN,NaN,12.225225,2192.983333,899.683333,13.769848


In [84]:
pb.to_csv('../../data/GO_pb_times.csv')

In [85]:
npb.head()

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
0,2014,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),9.541667,NaN,NaN,8.592105,NaN,10.600324,12.399177,17.733333,9.107843,14.792880,8.591837,17.929167,8.622222,10.630081,NaN,NaN,25.088435,11.418919,13.769231,21.262963,8.951667,12.495238,NaN,NaN,18.377581,NaN,NaN,15.748299,22.459119,NaN,13.636111,NaN,22.088745,NaN,NaN,18.590909,NaN,NaN,7.968468,2312.483333,1100.750000,14.183180
1,2011,003cc0338235bb1862eb7b9bdd31ec9c,Male,Senior Men (23-39),10.694853,NaN,NaN,10.660088,NaN,11.444984,14.790123,18.772727,10.356209,14.632686,9.476190,17.833333,11.033333,13.252033,NaN,NaN,25.986395,11.813063,16.256410,25.833333,15.586667,17.473810,NaN,21.075426,NaN,NaN,16.559829,NaN,21.588050,NaN,11.688889,NaN,NaN,10.767196,NaN,NaN,11.762452,NaN,NaN,2581.333333,1344.616667,15.188612
2,2008,006db776bb4048de7f357c632a8778ac,Male,Senior Men (23-39),NaN,4.723913,NaN,NaN,8.472222,9.040453,12.572016,16.218182,9.852941,15.737864,11.806122,21.025000,8.951852,8.869919,NaN,NaN,25.017007,19.596847,13.961538,26.729630,10.395000,14.508333,27.432660,NaN,NaN,21.005291,NaN,NaN,24.845912,NaN,17.430556,NaN,30.506494,NaN,NaN,15.212121,NaN,NaN,18.905405,2707.316667,1450.333333,16.367387
3,2016,00a7ade381ebb6d25e1aacb0bc7129e9,Male,Masters Men 1 (40-49),8.912990,NaN,NaN,7.186404,NaN,10.422330,13.958848,19.400000,11.287582,16.563107,26.071429,16.337500,10.370370,11.008130,48.0,NaN,22.217687,14.506757,17.227564,21.659259,11.713333,14.100000,NaN,NaN,22.088496,NaN,NaN,15.302721,25.490566,NaN,13.372222,NaN,25.939394,NaN,NaN,20.959596,NaN,NaN,15.957207,2703.916667,1383.883333,17.602140
4,2008,00b39183bebaffe426e376f7efc8ae1b,Male,Masters Men 1 (40-49),NaN,NaN,NaN,NaN,29.197222,10.140777,13.462963,17.600000,11.225490,14.611650,11.180272,17.779167,10.611111,10.662602,NaN,NaN,22.261905,12.840090,13.230769,20.633333,10.486667,11.557143,17.387205,NaN,NaN,15.240741,NaN,NaN,20.226415,NaN,13.408333,NaN,19.309524,NaN,NaN,11.808081,NaN,NaN,9.322072,2255.450000,998.466667,14.964501


# Define Functions

In [86]:
def find_similar_times(index):
    query = npb.iloc[index]
    same_gender = npb[npb.gender == query.gender]
    similar_times = same_gender[(same_gender.AVG_Pace > query.AVG_Pace - pace_threshold) & (same_gender.AVG_Pace < query.AVG_Pace + pace_threshold)]
    #find all runners either 0.5 minutes/kilometre faster or slower than the query runner
   # if len(similar_times) = 1:
    #    similar_times
    return similar_times
    #else:
     #   return None

# Show sample output
find_similar_times(66)

,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace
16,2011,025186c49c43c3cc6758fca4d331adb5,Male,Senior Men (23-39),8.158088,NaN,NaN,6.254386,NaN,7.210356,10.709877,14.657576,7.179739,13.019417,6.241497,16.104167,6.877778,7.947154,NaN,NaN,25.714286,9.407658,8.814103,17.618519,10.196667,10.744048,NaN,12.761557,NaN,NaN,17.547009,NaN,20.798742,NaN,10.213889,NaN,NaN,11.058201,NaN,NaN,9.597701,NaN,NaN,1938.866667,702.150000,11.688366
21,2011,0386a04fa05e91c6db0d57785c3f8801,Male,Masters Men 2 (50-59),9.705882,NaN,NaN,8.993421,NaN,9.766990,13.851852,16.424242,7.326797,14.297735,6.353741,18.220833,7.222222,12.495935,NaN,NaN,23.153061,9.700450,14.775641,19.029630,10.326667,14.815476,NaN,16.712895,NaN,NaN,19.104701,NaN,19.399371,NaN,8.322222,NaN,NaN,10.005291,NaN,NaN,8.252874,NaN,NaN,2199.816667,963.100000,12.967736
25,2015,04578c529d977b6912c3a413df104431,Male,Masters Men 1 (40-49),7.584559,NaN,NaN,5.574561,NaN,7.690939,9.358025,13.415152,7.359477,11.249191,15.829932,14.837500,7.233333,8.540650,42.944444,16.854167,NaN,8.873874,9.634615,17.081481,8.223333,10.917857,NaN,NaN,13.362832,NaN,NaN,8.187075,15.003145,NaN,6.633333,NaN,13.705628,NaN,NaN,10.292929,NaN,NaN,6.475225,1763.833333,494.583333,11.874530
31,2011,050587e57898af56e2b0f1fae0bb00bc,Male,Masters Men 2 (50-59),7.736520,NaN,NaN,6.592105,NaN,7.711974,9.174897,14.442424,8.179739,11.153722,6.962585,13.345833,8.144444,9.154472,NaN,NaN,16.350340,10.306306,10.221154,17.344444,11.898333,10.764286,NaN,16.347932,NaN,NaN,17.104701,NaN,18.358491,NaN,11.980556,NaN,NaN,10.116402,NaN,NaN,9.919540,NaN,NaN,1940.650000,703.933333,11.448313
43,2011,07dd98de0ea64ae2dfc7e2cfac8a1ead,Male,Masters Men 1 (40-49),7.623775,NaN,NaN,6.600877,NaN,7.252427,8.469136,14.181818,8.545752,11.059871,6.187075,13.912500,7.070370,8.215447,NaN,NaN,20.435374,9.907658,14.105769,26.303704,12.021667,10.690476,NaN,14.596107,NaN,NaN,19.711538,NaN,20.352201,NaN,9.263889,NaN,NaN,9.293651,NaN,NaN,9.350575,NaN,NaN,1976.233333,739.516667,11.963115
49,2011,0999c52336f41c6ee35be9fb5f01fbbd,Male,Masters Men 1 (40-49),7.674020,NaN,NaN,6.500000,NaN,7.826861,10.000000,16.412121,8.179739,12.637540,7.302721,15.125000,8.077778,9.325203,NaN,NaN,23.928571,10.164414,11.198718,18.840741,9.930000,11.295238,NaN,13.772506,NaN,NaN,15.861111,NaN,18.550314,NaN,10.530556,NaN,NaN,11.494709,NaN,NaN,9.262452,NaN,NaN,1968.216667,731.500000,11.908275
50,2009,0999c52336f41c6ee35be9fb5f01fbbd,Male,Masters Men 1 (40-49),NaN,NaN,7.547287,NaN,NaN,7.857605,11.407407,15.836364,9.460784,13.521036,7.704082,16.029167,8.262963,8.902439,NaN,NaN,25.534014,10.362613,11.448718,19.129630,8.578333,10.577381,15.693603,NaN,NaN,10.701058,NaN,NaN,17.547170,NaN,10.280556,NaN,18.590909,NaN,NaN,15.429293,NaN,NaN,9.468468,2008.650000,715.350000,12.603082
54,2013,09d9cc6f4f2b2669695eb7de6941ac6b,Male,Masters Men 1 (40-49),7.828431,NaN,NaN,5.502193,NaN,7.920712,9.333333,13.781818,7.454248,12.367314,7.309524,14.470833,8.529630,9.365854,NaN,NaN,21.680272,9.443694,10.092949,17.629630,7.723333,10.977381,15.552189,NaN,NaN,13.103175,NaN,NaN,20.144654,NaN,9.613889,NaN,21.796537,NaN,NaN,14.323232,NaN,NaN,17.637387,2002.5833

In [87]:
def cosine_similarity(a, b):
    # Combine two lists (row and query run) into matrix
    matrix = pd.DataFrame({"A": a, "B": b})
    # Now drop any pairs containing NA so we only compare when we have two times
    matrix = matrix.dropna(axis = 0, how='any')
    # Parse back to two lists
    a = matrix[['A']]
    b = matrix[['B']]
    # Return cosine similarity (1 - cosine difference)
    return 1 - (cosine(a, b))

def sims(index):
    similar_times = find_similar_times(index)
    #similar_times = similar_times.dropna(how='all')
    #if sim_count > 1:
    similar_times = similar_times.filter(like='to')
    #if similar_times.count() > 1:
    query_segs = list(similar_times.loc[index])
    similar_times = similar_times.drop([index], axis = 0)
    result = similar_times.apply(lambda row: cosine_similarity(list(row), query_segs), axis = 1)
    result = result.sort_values(ascending = False) #finding the most similar
    result = result.head(10) #10 is the number of neighbours as an example, the optimal value for n is tested in Prediction_Eval notebook
    
    return result

    #elif sim_count <= 1:
     #   return 0

sims(194)

945     0.997677
477     0.997125
1088    0.997106
450     0.996968
939     0.996961
1229    0.996863
1090    0.996678
17      0.996542
193     0.996161
940     0.996156
dtype: float64

In [88]:
def euc_distance(a, b):
    # Combine two lists (row and query run) into matrix
    matrix = pd.DataFrame({"A": a, "B": b})
    # Now drop any pairs containing NA so we only compare when we have two times
    matrix = matrix.dropna(axis = 0, how='any')
    # Parse back to two lists
    a = matrix[['A']]
    b = matrix[['B']]
    
    return euclidean(a,b)

def euc_sims(index):
    similar_times = find_similar_times(index)
    #similar_times = similar_times.dropna(how='all')
    #if sim_count > 1:
    similar_times = similar_times.filter(like='to')
    #if similar_times.count() > 1:
    query_segs = list(similar_times.loc[index])
    similar_times = similar_times.drop([index], axis = 0)
    result = similar_times.apply(lambda row: euc_distance(list(row), query_segs), axis = 1)
    result = result.sort_values(ascending = True) #finding the most similar
    result = result.head(10) #10 is the number of neighbours as an example, the optimal value for n is tested in Prediction_Eval notebook
    
    return result

    #elif sim_count <= 1:
     #   return 0

euc_sims(194)

1088    3.149025
1229    3.573367
477     3.637299
1090    3.907863
1596    3.933779
193     3.936973
815     4.055759
930     4.144093
195     4.241338
449     4.430873
dtype: float64

In [89]:
def weight(index):
    weighter = sims(index)
    #weight_count = len(weighter)
    #weight0 = len(str(weight_count))
   # if weight_count > 1:
   # if weighter.sum() > 3:
    if len(weighter) > 1:
        weighter = weighter / weighter.sum()
        return weighter
    
    else:
        weighter1 = weighter / weighter
        return weighter1 / 10
    
    #else:
     #   return np.nan
    
    #elif weight0 <= 1:
     #   return 0
    

weight(194)

945     0.100086
477     0.100030
1088    0.100028
450     0.100014
939     0.100014
1229    0.100004
1090    0.099985
17      0.099972
193     0.099934
940     0.099933
dtype: float64

In [90]:
def euc_weight(index):
    weighter = euc_sims(index)
    if len(weighter) > 1:
        weighter = 1 -(weighter / weighter.sum())
        weighter = weighter / weighter.sum()
        return weighter
    else:
        weighter1 = weighter / weighter
        return weighter1 / 10
    
euc_weight(194)

1088    0.102142
1229    0.100933
477     0.100751
1090    0.099981
1596    0.099907
193     0.099898
815     0.099559
930     0.099308
195     0.099031
449     0.098491
dtype: float64

In [91]:
def find_nearest_neighbours(index):
    #w = weight(index)
    cosine_distance = weight(index)  
    nearest_neighbours = merge.iloc[cosine_distance.index]
    if len(nearest_neighbours) > 1:
        nearest_neighbours['weights'] = weight(index)
        nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb']
   # nearest_neighbours['rank_weights'] = rank_weight(index)
   # nearest_neighbours['ranked_pace'] = nearest_neighbours['rank_weights'] * nearest_neighbours['AVG_Pace_pb']
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
        return nearest_neighbours
    else: 
        nearest_neighbours['weights'] = weight(index) * 10
        nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb']
        
        return nearest_neighbours
    
# Show sample output    
find_nearest_neighbours(194)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,gender_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb,weights,weighted_pace
945,2013,93a490693d00fa5854293ff7be213a34,Male,Senior Men (23-39),6.183824,NaN,NaN,5.050439,NaN,6.444984,7.345679,12.357576,6.552288,9.632686,5.918367,12.295833,7.051852,8.471545,NaN,NaN,14.823129,9.148649,8.342949,14.422222,6.700000,8.123810,9.538721,NaN,NaN,7.968254,NaN,NaN,12.933962,NaN,6.302778,NaN,12.153680,NaN,NaN,7.954545,NaN,NaN,6.995495,1445.566667,210.616667,8.863053,Male,5.610294,NaN,NaN,4.609649,NaN,6.059871,7.234568,11.306061,5.552288,8.828479,12.166667,11.358333,6.018519,7.105691,9.666667,14.833333,NaN,8.297297,7.961538,12.596296,5.750000,7.401190,NaN,NaN,9.840708,NaN,NaN,6.384354,13.698113,NaN,6.427778,NaN,14.538961,NaN,NaN,10.848485,NaN,NaN,8.081081,1424.883333,155.633333,8.887049,0.100086,0.889466
477,2009,4bcd7473cdbc13d04aa01409261a202d,Male,Senior Men (23-39),NaN,NaN,5.433333,NaN,NaN,6.119741,7.771605,11.339394,6.807190,9.433657,5.568027,10.995833,6.685185,7.934959,NaN,NaN,13.571429,8.560811,7.538462,12.466667,6.258333,7.489286,9.309764,NaN,NaN,7.283069,NaN,NaN,12.613208,NaN,5.844444,NaN,11.759740,NaN,NaN,7.601010,NaN,NaN,5.085586,1356.750000,63.450000,8.411771,Male,5.938725,NaN,NaN,4.866228,NaN,5.598706,6.917695,10.303030,5.758170,8.430421,5.112245,9.512500,5.511111,5.975610,NaN,NaN,11.374150,6.885135,6.455128,11.781481,6.431667,6.904762,NaN,7.543796,NaN,NaN,11.096154,NaN,11.201258,NaN,5.452778,NaN,NaN,6.798942,NaN,NaN,6.885057,NaN,NaN,1255.683333,18.966667,7.510206,0.100030,0.751248
1088,2017,a82e067a0e3f059af79e29cad5b6b70f,Male,Senior Men (23-39),5.466912,NaN,NaN,4.453947,NaN,5.980583,7.312757,11.381818,5.575163,9.645631,4.795918,11.587500,6.188889,6.556911,NaN,NaN,12.425170,8.921171,8.496795,14.466667,6.020000,7.153571,NaN,NaN,10.405605,NaN,NaN,7.707483,NaN,11.309028,NaN,10.284483,NaN,NaN,9.626126,NaN,NaN,10.490476,5.858108,1362.216667,220.316667,8.421280,Male,5.604167,NaN,NaN,4.394737,NaN,6.129450,6.802469,11.133333,5.807190,8.053398,4.479592,9.779167,5.359259,5.914634,NaN,NaN,12.119048,7.141892,6.945513,12.296296,6.226667,7.544048,9.621212,NaN,NaN,8.023

In [92]:
def euc_find_nearest_neighbours(index):
    #w = weight(index)
    euc_distance = euc_weight(index)  
    nearest_neighbours = merge.iloc[euc_distance.index]
    if len(nearest_neighbours) > 1: 
        nearest_neighbours['weights'] = euc_weight(index)
        nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb']
   # nearest_neighbours['rank_weights'] = rank_weight(index)
   # nearest_neighbours['ranked_pace'] = nearest_neighbours['rank_weights'] * nearest_neighbours['AVG_Pace_pb']
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
        return nearest_neighbours
    
    else: 
        nearest_neighbours['weights'] = euc_weight(index) * 10
        nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb'] 
   # nearest_neighbours['rank_weights'] = rank_weight(index)
   # nearest_neighbours['ranked_pace'] = nearest_neighbours['rank_weights'] * nearest_neighbours['AVG_Pace_pb']
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
        return nearest_neighbours
    
# Show sample output    
euc_find_nearest_neighbours(194)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,Year,name_mask,gender,category,Chamonix to Delevret,Chamonix to La Charme,Chamonix to St-Gervais,Delevret to St-Gervais,La Charme to St-Gervais,St-Gervais to Contamines,Contamines to La Balme,La Balme to Bonhomme,Bonhomme to Chapieux,Chapieux to Col Seigne,Col Seigne to Lac Combal,Lac Combal to Mt-Favre,Mt-Favre to Checruit,Checruit to Courmayeur,Courmayeur to Courmayeur2,Courmayeur2 to Bertone,Courmayeur to Bertone,Bertone to Bonatti,Bonatti to Arnouvaz,Arnouvaz to Col Ferret,Col Ferret to La Fouly,La Fouly to Champex La,Champex La to Bovine,Champex La to Mortigny,Champex La to Giete,Bovine to Trient,Mortigny to Trient,Giete to Trient,Trient to La Catogne,Trient to Les Tseppe,La Catogne to Vallorcine,Les Tseppe to Vallorcine,Vallorcine to Tete aux Vents,Vallorcine to Argentiere,Vallorcine to Col Montet,Tete aux Vents to Flegere,Argentiere to Chamonix,Col Montet to Flegere,Flegere to Chamonix,time,timediff,AVG_Pace,gender_pb,Chamonix to Delevret_pb,Chamonix to La Charme_pb,Chamonix to St-Gervais_pb,Delevret to St-Gervais_pb,La Charme to St-Gervais_pb,St-Gervais to Contamines_pb,Contamines to La Balme_pb,La Balme to Bonhomme_pb,Bonhomme to Chapieux_pb,Chapieux to Col Seigne_pb,Col Seigne to Lac Combal_pb,Lac Combal to Mt-Favre_pb,Mt-Favre to Checruit_pb,Checruit to Courmayeur_pb,Courmayeur to Courmayeur2_pb,Courmayeur2 to Bertone_pb,Courmayeur to Bertone_pb,Bertone to Bonatti_pb,Bonatti to Arnouvaz_pb,Arnouvaz to Col Ferret_pb,Col Ferret to La Fouly_pb,La Fouly to Champex La_pb,Champex La to Bovine_pb,Champex La to Mortigny_pb,Champex La to Giete_pb,Bovine to Trient_pb,Mortigny to Trient_pb,Giete to Trient_pb,Trient to La Catogne_pb,Trient to Les Tseppe_pb,La Catogne to Vallorcine_pb,Les Tseppe to Vallorcine_pb,Vallorcine to Tete aux Vents_pb,Vallorcine to Argentiere_pb,Vallorcine to Col Montet_pb,Tete aux Vents to Flegere_pb,Argentiere to Chamonix_pb,Col Montet to Flegere_pb,Flegere to Chamonix_pb,time_pb,timediff_pb,AVG_Pace_pb,weights,weighted_pace
1088,2017,a82e067a0e3f059af79e29cad5b6b70f,Male,Senior Men (23-39),5.466912,NaN,NaN,4.453947,NaN,5.980583,7.312757,11.381818,5.575163,9.645631,4.795918,11.587500,6.188889,6.556911,NaN,NaN,12.425170,8.921171,8.496795,14.466667,6.020000,7.153571,NaN,NaN,10.405605,NaN,NaN,7.707483,NaN,11.309028,NaN,10.284483,NaN,NaN,9.626126,NaN,NaN,10.490476,5.858108,1362.216667,220.316667,8.421280,Male,5.604167,NaN,NaN,4.394737,NaN,6.129450,6.802469,11.133333,5.807190,8.053398,4.479592,9.779167,5.359259,5.914634,NaN,NaN,12.119048,7.141892,6.945513,12.296296,6.226667,7.544048,9.621212,NaN,NaN,8.023810,NaN,NaN,14.267296,NaN,7.458333,NaN,12.790043,NaN,NaN,8.186869,NaN,NaN,7.279279,1328.183333,93.233333,8.056571,0.102142,0.822914
1229,2017,bf5f4fb86e38559cba1d9b622aa26fd4,Male,Senior Men (23-39),5.631127,NaN,NaN,4.969298,NaN,6.027508,7.008230,11.196970,7.222222,9.572816,6.006803,11.704167,7.911111,8.060976,NaN,NaN,13.003401,8.168919,7.233974,14.081481,5.973333,6.442857,NaN,NaN,8.873156,NaN,NaN,6.853741,NaN,9.430556,NaN,9.132184,NaN,NaN,8.572072,NaN,NaN,11.621429,6.083333,1340.366667,198.466667,8.365903,Male,5.894608,NaN,NaN,5.118421,NaN,5.998382,6.769547,11.630303,6.777778,8.820388,14.207483,11.233333,6.625926,7.617886,4.314815,15.4,NaN,7.540541,7.858974,12.029630,5.881667,7.025000,NaN,NaN,7.756637,NaN,NaN,5.935374,10.798742,NaN,5.530556,NaN,10.588745,NaN,NaN,7.348485,NaN,NaN,4.412162,1319.700000,50.450000,8.124615,0.100933,0.820044
477,2009,4bcd7473cdbc13d04aa01409261a202d,Male,Senior Men (23-39),NaN,NaN,5.433333,NaN,NaN,6.119741,7.771605,11.339394,6.807190,9.433657,5.568027,10.995833,6.685185,7.934959,NaN,NaN,13.571429,8.560811,7.538462,12.466667,6.258333,7.489286,9.309764,NaN,NaN,7.283069,NaN,NaN,12.613208,NaN,5.844444,NaN,11.759740,NaN,NaN,7.601010,NaN,NaN,5.085586,1356.750000,63.450000,8.411771,Male,5.938725,NaN,NaN,4.866228,NaN,5.598706,6.917695,10.303030,5.758170,8.430421,5.112245,9.512500,5.511111,5.975610,NaN,NaN,11.374150,6.885135,6.455128,11.781481,6.431667,6.904762,NaN,7.543796,NaN,NaN,11.

####  means

In [93]:
def get_mean_pb(neighbours):
    mean = neighbours.AVG_Pace_pb.mean() 
    return mean
    
# Show sample output    
neighbours = find_nearest_neighbours(10)
get_mean_pb(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


15.246587226028856

In [94]:
def get_weight_mean(neighbours):
    w_mean = neighbours.weighted_pace.sum()
    return w_mean

neighbours = find_nearest_neighbours(10)
get_weight_mean(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


15.246259762358578

In [95]:
def euc_get_weight_mean(neighbours):
    ew_mean = neighbours.weighted_pace.sum()
    return ew_mean

neighbours = euc_find_nearest_neighbours(10)
euc_get_weight_mean(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


15.267969152245048

In [96]:
def get_best_npb(neighbours):
    return neighbours.AVG_Pace.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(60)
get_best_npb(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


9.6221401970722837

## Get Best

In [97]:
def get_best_pb(neighbours):
    return neighbours.AVG_Pace_pb.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(5)
get_best_pb(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


14.020416541697346

In [98]:
def get_prediction(index, neighbours):
    weight = merge.iloc[index].AVG_Pace/get_best_npb(neighbours)
    best_pb = get_best_pb(neighbours)
    return best_pb * weight

# Show output
neighbours = find_nearest_neighbours(0)
get_prediction(0, neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


12.994630049037474

# Calculate Predictions

In [99]:
def update_rows(row):
    index = int(row['best_pb_prediction'])
    row['query_AVG_Pace'] = merge.iloc[index].AVG_Pace
    row['query_pb'] = merge.iloc[index].AVG_Pace_pb
    neighbours = find_nearest_neighbours(index)
    row['mean_pb_average_pace'] = get_mean_pb(neighbours)
   # row['weighted_mean_pb_ap'] = weighted_pace(neighbours)
  #  row['best_average_pace'] = get_best_npb(neighbours)
    row['best_pb_prediction'] = get_prediction(index, neighbours)
    row['weighted_mean'] = get_weight_mean(neighbours)
    neighbours1 = euc_find_nearest_neighbours(index)
    row['euc_weighted_mean'] = euc_get_weight_mean(neighbours1)
   # row['ranked_mean'] = get_rank_mean(neighbours)
    return row

In [100]:
num_of_runs = merge.time.count()
filler_data = np.arange(0.0, num_of_runs)
df = pd.DataFrame({'best_pb_prediction':filler_data, 'mean_pb_average_pace':filler_data,  'weighted_mean':filler_data,   'euc_weighted_mean':filler_data, 'query_AVG_Pace':filler_data, 'query_pb':filler_data})

df = df.apply(lambda row: update_rows(row), axis=1)
df = df[['query_AVG_Pace', 'query_pb', 'best_pb_prediction', 'mean_pb_average_pace', 'weighted_mean', 'euc_weighted_mean']]
df

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

,query_AVG_Pace,query_pb,best_pb_prediction,mean_pb_average_pace,weighted_mean,euc_weighted_mean
0,14.183180,13.489501,12.994630,13.267151,13.267170,13.084611
1,15.188612,13.489501,14.561941,15.165493,15.165590,14.544371
2,16.367387,13.945114,14.862793,15.946644,15.946630,16.087892
3,17.602140,15.121991,14.592674,16.020199,16.020118,15.783054
4,14.964501,13.769848,13.358058,14.176924,14.177359,13.047495
5,16.136556,16.626208,14.915325,15.004537,15.005013,15.323115
6,14.533689,13.438194,10.894412,13.329228,13.328519,13.252288
7,16.813847,18.265522,14.647014,15.719932,15.719859,15.561262
8,15.182644,11.779689,13.637777,14.452719,14.452918,14.558441
9,16.945035,14.825623,15.079635,15.991020,15.991015,16.390487


In [101]:
df.iloc[[66]]

,query_AVG_Pace,query_pb,best_pb_prediction,mean_pb_average_pace,weighted_mean,euc_weighted_mean
66,12.404403,11.110489,9.845431,12.140451,12.140227,12.094083


Comb dataframe contains the information of the gender and category of each runner. This will be used in a later notebook to test how accurate the predictions are in comparison to male, female, senior men, etc...

In [102]:
comb.join(df)

,category,gender,query_AVG_Pace,query_pb,best_pb_prediction,mean_pb_average_pace,weighted_mean,euc_weighted_mean
0,Senior Men (23-39),Male,14.183180,13.489501,12.994630,13.267151,13.267170,13.084611
1,Senior Men (23-39),Male,15.188612,13.489501,14.561941,15.165493,15.165590,14.544371
2,Senior Men (23-39),Male,16.367387,13.945114,14.862793,15.946644,15.946630,16.087892
3,Masters Men 1 (40-49),Male,17.602140,15.121991,14.592674,16.020199,16.020118,15.783054
4,Masters Men 1 (40-49),Male,14.964501,13.769848,13.358058,14.176924,14.177359,13.047495
5,Masters Men 1 (40-49),Male,16.136556,16.626208,14.915325,15.004537,15.005013,15.323115
6,Masters Men 1 (40-49),Male,14.533689,13.438194,10.894412,13.329228,13.328519,13.252288
7,Senior Men (23-39),Male,16.813847,18.265522,14.647014,15.719932,15.719859,15.561262
8,Senior Men (23-39),Male,15.182644,11.779689,13.637777,14.452719,14.452918,14.558441
9,Senior Men (23-39),Male,16.945035,14.825623,15.079635,15.991020,15.991015,16.390487


In [103]:
df = comb.join(df)

In [104]:
df.sort_values(by = 'weighted_mean')

,category,gender,query_AVG_Pace,query_pb,best_pb_prediction,mean_pb_average_pace,weighted_mean,euc_weighted_mean
193,Senior Men (23-39),Male,7.416191,7.097568,7.020033,7.636395,7.636226,7.598918
815,Senior Men (23-39),Male,7.408082,7.012357,7.089808,7.708039,7.707968,7.633907
195,Youth Men (20-22),Male,7.599352,7.097568,7.193410,7.846926,7.846976,7.708483
930,Masters Men 1 (40-49),Male,7.612542,7.453740,7.205895,7.962389,7.962297,7.651181
1090,Senior Men (23-39),Male,8.235962,8.056571,7.796014,8.001424,8.001417,8.100525
1596,Senior Men (23-39),Male,8.039194,8.141435,7.609757,8.006512,8.006348,7.995451
1231,Senior Men (23-39),Male,8.250002,8.106394,7.815027,8.009311,8.009133,7.927700
948,Senior Men (23-39),Male,7.761441,7.466542,7.496033,8.043536,8.043398,7.780287
1229,Senior Men (23-39),Male,8.365903,8.124615,7.813504,8.072049,8.072043,8.176126
1595,Senior Men (23-39),Male,8.140912,8.141435,7.711689,8.089196,8.088658,8.079071


In [105]:
df.to_csv('../../data/GO_pace_predictions.csv')

In [109]:
df.query_pb.mean()

13.711955343250013

# Coverage

In [106]:
predict = pd.read_csv('../../data/GO_pace_predictions.csv')

In [107]:
predict = len(predict)

In [108]:
(predict / first_df) * 100

100.0